In [ ]:
from pyecsca.ec.model import ShortWeierstrassModel, MontgomeryModel, TwistedEdwardsModel
from pyecsca.ec.formula_gen.test import load_efd_formulas, load_library_formulas
from pyecsca.ec.formula_gen.formula_graph import EFDFormulaGraph
from pyecsca.ec.formula_gen.fliparoo import generate_fliparood_formulas, greedy_fliparoo, recursive_fliparoo
import pyecsca.ec.formula_gen.metrics as metrics
from pyecsca.ec.formula_gen.switch_sign import generate_switched_formulas
from tqdm.notebook import tqdm

In [ ]:
# Draw formula
coordinate_name = "jacobian"
model = ShortWeierstrassModel()
name = "add-1998-cmo-2"
formula = load_efd_formulas(coordinate_name,model)[name]
graph = EFDFormulaGraph()
graph.construct_graph(formula)
graph.draw()

In [ ]:
# Draw all formulas
coordinate_name = "jacobian"
model = ShortWeierstrassModel()
for name, formula in load_efd_formulas(coordinate_name,model).items():
    graph = EFDFormulaGraph()
    graph.construct_graph(formula)
    graph.draw(f"{coordinate_name}:{name}.png")

In [ ]:
# Measure similarity of fliparood formulas
coordinate_name = "jacobian"
model = ShortWeierstrassModel()
name = "add-1998-cmo-2"
formula = load_efd_formulas(coordinate_name,model)[name]
for fliparood in generate_fliparood_formulas(formula):
    print(metrics.formula_similarity(formula,fliparood))

In [ ]:
# Greedy fliparoo to connect two formulas
coordinate_name = "jacobian"
model = ShortWeierstrassModel()
name = "add-1998-cmo-2"
formula = load_efd_formulas(coordinate_name,model)[name]
lib_formula = load_library_formulas()["add-openssl-z256"]
metric = lambda x: metrics.formula_similarity(x,formula)["ivs"]
flips, closest, sim = greedy_fliparoo(lib_formula,metric)
print(f"Number of flips: {flips}, similarity: {sim}")

In [ ]:
# Generate all fliparoos for all library formulas
depth = 2
fliparood = {}
libs = load_library_formulas()
for name, formula in tqdm(libs.items()):
    fliparood[name] = recursive_fliparoo(formula,depth)


In [ ]:
""" Connect the generated formulas to efd """
neighborhoods = fliparood 
for name,formulas in fliparood.items():
    coordinates, model = libs[name].coordinate_model.name, libs[name].coordinate_model.curve_model
    similarities = {}
    for efd_name, efd_formula in load_efd_formulas(coordinates, model.__class__).items():
        metric = lambda x: metrics.formula_similarity(x,efd_formula)["ivs"]
        flips, closest_fliparoo = max(formulas, key = lambda x: metric(x[1]))
        similarities[efd_name] = flips, metric(closest_fliparoo)
    closest_efd, (flips, sim) = max(similarities.items(), key = lambda x: x[1][1])
    print(f"{name}. Closest match: {closest_efd}, flips={flips}, similarity={sim}")


In [ ]:
switch_signed = {}
for name, fliparoo_neighborhood in tqdm(fliparood.items()):
    neighb = list()
    for flips,flip_f in fliparoo_neighborhood:
        neighb.extend(generate_switched_formulas(flip_f))
    switch_signed[name] = neighb

In [ ]:
""" Connect the generated formulas to efd """
neighborhoods = switch_signed 
for name,formulas in tqdm(switch_signed.items()):
    coordinates, model = libs[name].coordinate_model.name, libs[name].coordinate_model.curve_model
    similarities = {}
    for efd_name, efd_formula in load_efd_formulas(coordinates, model.__class__).items():
        metric = lambda x: metrics.formula_similarity(x,efd_formula)["ivs"]
        closest_switch_signed = max(formulas, key = metric)
        similarities[efd_name] = metric(closest_switch_signed)
    closest_efd, sim = max(similarities.items(), key = lambda x: x[1])
    print(f"{name}. Closest match: {closest_efd}, similarity={sim}")